In [59]:
import numpy as np
import pandas as pd
from scipy import stats

In [60]:
pd.options.display.float_format = "{:_.6f}".format

In [61]:
df = (
    pd.read_excel(
        "data/full/total_votos.xlsx",
        header=[1, 2],
        index_col=[0, 1, 2],
        na_values=["-"],
        sheet_name="Plan1",
    )
    .dropna(how='all')
    .rename(columns={
        '1 Turno': 1,
        '2 Turno ': 2
    })
    .rename_axis(['regiao', 'grupo', 'uf'], axis=0)
    .rename_axis(['cargo', 'turno'], axis=1)
)
df

cargo                 Governador/Senador                          Presidente  \
turno                                  1                 2                 1   
regiao       grupo uf                                                          
Sudeste      1     ES   2_311_629.000000  2_318_350.000000  2_315_889.000000   
                   MG  12_643_691.000000               NaN 12_655_228.000000   
                   SP  27_147_847.000000 27_341_816.000000 27_189_714.000000   
                   RJ   9_893_658.000000               NaN  9_909_463.000000   
Sul          2     PR   6_815_688.000000               NaN  6_828_543.000000   
                   RS   6_883_341.000000  6_924_603.000000  6_890_016.000000   
                   SC   4_471_619.000000  4_527_781.000000  4_487_474.000000   
Centro-Oeste 3     DF   1_807_484.000000               NaN  1_819_900.000000   
                   GO   3_807_191.000000               NaN  3_812_597.000000   
                   MS   1_552_654.000000  1_547_610.000000  1_555_149.000000   
                   MT   1_889_012.000000               NaN  1_892_180.000000   
Nordeste     4     AL   1_802_791.000000  1_782_407.000000  1_805_971.000000   
                   BA   8_866_459.000000  8_964_244.000000  8_874_841.000000   
                   SE   1_362_604.000000  1_353_538.000000  1_364_724.000000   
             5     CE   5_623_450.000000               NaN  5_628_610.000000   
                   MA   3_918_212.000000               NaN  3_920_435.000000   
                   PI   2_114_006.000000               NaN  2_115_645.000000   
             6     PB   2_551_876.000000  2_569_106.000000  2_557_479.000000   
                   PE   5_732_289.000000  5_795_359.000000  5_738_371.000000   
                   RN   2_086_722.000000               NaN  2_090_604.000000   
Norte        7     AC     455_438.000000               NaN    455_903.000000   
                   RO     925_763.000000    925_563.000000    926_827.000000   
             8     AM   2_110_875.000000  2_065_079.000000  2_113_771.000000   
                   RR     304_319.000000               NaN    305_404.000000   
             9     AP     442_101.000000               NaN    442_842.000000   
                   PA   4_785_272.000000               NaN  4_789_311.000000   
                   TO     889_580.000000               NaN    891_449.000000   

cargo                                    
turno                                 2  
regiao       grupo uf                    
Sudeste      1     ES  2_322_269.000000  
                   MG 12_866_284.000000  
                   SP 27_380_491.000000  
                   RJ  9_973_822.000000  
Sul          2     PR  6_900_420.000000  
                   RS  6_930_852.000000  
                   SC  4_542_817.000000  
Centro-Oeste 3     DF  1_838_492.000000  
                   GO  3_860_351.000000  
                   MS  1_549_873.000000  
                   MT  1_913_231.000000  
Nordeste     4     AL  1_784_988.000000  
                   BA  8_971_728.000000  
                   SE  1_355_467.000000  
             5     CE  5_649_398.000000  
                   MA  3_854_804.000000  
                   PI  2_088_530.000000  
             6     PB  2_574_215.000000  
                   PE  5_800_735.000000  
                   RN  2_108_799.000000  
Norte        7     AC    420_760.000000  
                   RO    926_517.000000  
             8     AM  2_067_875.000000  
                   RR    286_269.000000  
             9     AP    400_683.000000  
                   PA  4_701_740.000000  
                   TO    871_238.000000

In [62]:
B = .02
z = stats.norm.ppf(1 - .05 / 2)

# Turno 1

In [63]:
Nh = df.loc["Norte", ('Presidente', 1)]

est = (
    df.loc["Norte", [('Presidente', 1)]]
    .assign(
        Nh = Nh,
        Wh = Nh / Nh.sum(),
        Ph = 1/2,
        Qh = 1/2
    )
)

In [75]:
df.loc["Norte", ('Presidente', 1)].sum()

np.float64(9925507.0)

In [64]:
est

cargo          Presidente               Nh       Wh       Ph       Qh
turno                   1                                            
grupo uf                                                             
7     AC   455_903.000000   455_903.000000 0.045932 0.500000 0.500000
      RO   926_827.000000   926_827.000000 0.093378 0.500000 0.500000
8     AM 2_113_771.000000 2_113_771.000000 0.212964 0.500000 0.500000
      RR   305_404.000000   305_404.000000 0.030770 0.500000 0.500000
9     AP   442_842.000000   442_842.000000 0.044617 0.500000 0.500000
      PA 4_789_311.000000 4_789_311.000000 0.482526 0.500000 0.500000
      TO   891_449.000000   891_449.000000 0.089814 0.500000 0.500000

In [65]:
s = est.apply(lambda e: e['Nh'] ** 2 * e['Ph'] * e['Qh'] / (e['Nh'] - 1), axis=1).sum()


In [66]:
N = Nh.sum()
D = (B / z) ** 2
n = np.ceil(
    N /
    ((N ** 2 * D) / s) + 1
)
n

turno
   2_402.000000
dtype: float64

In [67]:
est = est.assign(nh = lambda df_: (df_['Wh'] * n.values[0]).round())
est[['Nh', 'Wh', 'nh']]

cargo                  Nh       Wh           nh
turno                                          
grupo uf                                       
7     AC   455_903.000000 0.045932   110.000000
      RO   926_827.000000 0.093378   224.000000
8     AM 2_113_771.000000 0.212964   512.000000
      RR   305_404.000000 0.030770    74.000000
9     AP   442_842.000000 0.044617   107.000000
      PA 4_789_311.000000 0.482526 1_159.000000
      TO   891_449.000000 0.089814   216.000000

# Turno 2

In [68]:
Nh = df.loc["Norte", ('Presidente', 2)]

est = (
    df.loc["Norte", [('Presidente', 2)]]
    .assign(
        Nh = Nh,
        Wh = Nh / Nh.sum(),
        Ph = 1/2,
        Qh = 1/2
    )
)

In [73]:
df.loc["Norte", ('Presidente', 2)].sum()

np.float64(9675082.0)

In [69]:
est

cargo          Presidente               Nh       Wh       Ph       Qh
turno                   2                                            
grupo uf                                                             
7     AC   420_760.000000   420_760.000000 0.043489 0.500000 0.500000
      RO   926_517.000000   926_517.000000 0.095763 0.500000 0.500000
8     AM 2_067_875.000000 2_067_875.000000 0.213732 0.500000 0.500000
      RR   286_269.000000   286_269.000000 0.029588 0.500000 0.500000
9     AP   400_683.000000   400_683.000000 0.041414 0.500000 0.500000
      PA 4_701_740.000000 4_701_740.000000 0.485964 0.500000 0.500000
      TO   871_238.000000   871_238.000000 0.090050 0.500000 0.500000

In [70]:
s = est.apply(lambda e: e['Nh'] ** 2 * e['Ph'] * e['Qh'] / (e['Nh'] - 1), axis=1).sum()


In [71]:
N = Nh.sum()
D = (B / z) ** 2
n = np.ceil(
    N /
    ((N ** 2 * D) / s) + 1
)
n

turno
   2_402.000000
dtype: float64

In [72]:
est = est.assign(nh = lambda df_: (df_['Wh'] * n.values[0]).round())
est[['Nh', 'Wh', 'nh']]

cargo                  Nh       Wh           nh
turno                                          
grupo uf                                       
7     AC   420_760.000000 0.043489   104.000000
      RO   926_517.000000 0.095763   230.000000
8     AM 2_067_875.000000 0.213732   513.000000
      RR   286_269.000000 0.029588    71.000000
9     AP   400_683.000000 0.041414    99.000000
      PA 4_701_740.000000 0.485964 1_167.000000
      TO   871_238.000000 0.090050   216.000000